<a href="https://colab.research.google.com/github/mamoonmasud/graph_community/blob/main/GAT(cora).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [ ]:
#Checking if cuda is available. If return False, change runtime to GPU
print(torch.cuda.is_available())

In [3]:
print(torch.version.cuda)

11.1


In [4]:
#Checking Python Version
! python --version

Python 3.7.12


In [2]:
# Installing DGL
! pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html 

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 165.0 MB 52 kB/s 


In [3]:
from google.colab import files

In [5]:
# Doing all the imports:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
from scipy import sparse as sp
from math import log
import pandas as pd
from sklearn import metrics as skmetrics
import pickle
import numpy as np

import itertools

In [12]:
#from dgl.nn.pytorch.conv import GATConv as GAT

In [6]:
def upload_files():
  from google.colab import files
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(k, 'wb').write(v)
  return list(uploaded.keys())

In [7]:
upload_files()

Saving GAT.py to GAT.py
Saving performance.py to performance.py
Saving Sparsemax.py to Sparsemax.py


['GAT.py', 'performance.py', 'Sparsemax.py']

In [8]:
import GAT
import Sparsemax
import performance as pf


# Graph Neural Network Definition



In [9]:
class GATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim):
        super(GATLayer, self).__init__()
        self.g = g
        # equation (1)
        self.fc = nn.Linear(in_dim, out_dim, bias=False)
        # equation (2)
        self.attn_fc = nn.Linear(2 * out_dim, 1, bias=False)

    def edge_attention(self, edges):
        # edge UDF for equation (2)
        z2 = th.cat([edges.src['z'], edges.dst['z']], dim=1)
        a = self.attn_fc(z2)
        return {'e': F.leaky_relu(a)}

    def message_func(self, edges):
        # message UDF for equation (3) & (4)
        return {'z': edges.src['z'], 'e': edges.data['e']}

    def reduce_func(self, nodes):
        # reduce UDF for equation (3) & (4)
        # equation (3)
        alpha = F.softmax(nodes.mailbox['e'], dim=1)
        # equation (4)
        h = th.sum(alpha * nodes.mailbox['z'], dim=1)
        return {'h': h}

    def forward(self, h):
        # equation (1)
        z = self.fc(h)
        self.g.ndata['z'] = z
        # equation (2)
        self.g.apply_edges(self.edge_attention)
        # equation (3) & (4)
        self.g.update_all(self.message_func, self.reduce_func)
        return self.g.ndata.pop('h')


In [10]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, num_heads, merge='cat'):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            return th.cat(head_outs, dim=1)
        else:
            # merge using average
            return th.mean(torch.stack(head_outs))

In [11]:
class Net(nn.Module):
    def __init__(self, g, in_dim, hidden_dim, out_dim, num_heads):
        super(Net, self).__init__()
        self.layer1 = MultiHeadGATLayer(g, in_dim, hidden_dim, num_heads)
        # Be aware that the input dimension is hidden_dim*num_heads since
        # multiple head outputs are concatenated together. Also, only
        # one attention head in the output layer.
        self.layer2 = MultiHeadGATLayer(g, hidden_dim * num_heads, out_dim, 1)

    def forward(self, h):
        h = self.layer1(h)
        h = F.elu(h)
        h = self.layer2(h)
        h = F.log_softmax(h, 1)
        return h

# Data Loading

In [17]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()

# features = th.FloatTensor(data.features)
# labels = th.LongTensor(data.labels)
# mask = th.ByteTensor(data.train_mask)
#g1 = data.graph

##
g = data[0]
num_class = data.num_classes

features = g.ndata['feat']

train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
labels = g.ndata['label']

# add self loop
#g.remove_edges_from(nx.selfloop_edges(g))
g  = dgl.remove_self_loop(g)
#g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

comb_mask = torch.zeros(len(train_mask), dtype=torch.bool)

for i in range(len(comb_mask)):
    if test_mask[i] or train_mask[i] or val_mask[i]:
        comb_mask[i] = True


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [14]:
print('We have %d nodes.' % g.number_of_nodes())
print('We have %d edges.' % g.number_of_edges())

We have 2708 nodes.
We have 13264 edges.


# Selecting Training Set

In [15]:
percentage_train = 0.1
percentage_val = 0.1

with open("drive/MyDrive/CSCE689/data/cora_permutation3.pickle","rb") as f:
    perm1 = pickle.load(f)
mask_train = np.zeros(g.number_of_nodes())
mask_val = np.zeros(g.number_of_nodes())

i_train = int(percentage_train*g.number_of_nodes())
i_val = i_train + int(percentage_val*g.number_of_nodes())
mask_train[perm1[range(0,i_train)]] = 1
mask_val[perm1[range(i_train,i_val)]] = 1
mask_train = th.BoolTensor(mask_train)
mask_val = th.BoolTensor(mask_val)

## Training

In [18]:
loss_function = pf.perm_inv_loss(labels)
import copy

In [19]:
import time

net = GAT.GAT_Net_fast(g=g, in_feats=features.shape[1], hidden_size=100, hidden_layers=2, out_feats=len(np.unique(labels)),
dropout=0.2, batchnorm=False, num_heads=1, residual = False)
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-2, weight_decay=1e-2)
net.train() # Set to training mode (use dropout)

dur = []
loss_ev = []
current_best = 0 #arbitrarily high
current_best_epoch = 0
current_best_params = None
no_improvement_for = 0

for epoch in range(10000):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(features)
    train_rand=pf.rand_score(labels[mask_train].numpy(),np.argmax(prediction[mask_train].detach().numpy(), axis=1))
    validation_rand=pf.rand_score(labels[mask_val].numpy(),np.argmax(prediction[mask_val].detach().numpy(), axis=1))
    if train_rand>current_best:
        current_best = train_rand
        current_best_epoch = epoch
        current_best_params = copy.deepcopy(net.state_dict())
        no_improvement_for = 0
    else: no_improvement_for += 1
    
    if no_improvement_for>100:
        break
    
    net.train()

    # Compute loss for train nodes
    logits = net(features)

    #loss = loss_function.approximate_loss(logits,mask_train,nclasses=7)
    loss = F.nll_loss(logits[mask_train], labels[mask_train])
    loss_ev.append(loss.detach().item())
    #print(np.unique(np.argmax(logits[mask_train].detach().numpy(),1)))
    
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)
        print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | Train.Rand {train_rand:.4f} | Valid.Rand {validation_rand:.4f} | Time(s) {np.mean(dur):.4f}")
    else:
        print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | Train.Rand {train_rand:.4f} | Valid.Rand {validation_rand:.4f} | Time(s) unknown")
        
net.load_state_dict(current_best_params)

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00000 | Loss 3.6732 | Train.Rand 0.0824 | Valid.Rand 0.0905 | Time(s) unknown
Epoch 00001 | Loss 1.1571 | Train.Rand 0.4782 | Valid.Rand 0.3784 | Time(s) unknown


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00002 | Loss 0.7583 | Train.Rand 0.6778 | Valid.Rand 0.4908 | Time(s) unknown
Epoch 00003 | Loss 0.6842 | Train.Rand 0.7634 | Valid.Rand 0.5421 | Time(s) 0.1978


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


...

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00172 | Loss 0.1094 | Train.Rand 0.9884 | Valid.Rand 0.5842 | Time(s) 0.1799
Epoch 00173 | Loss 0.0647 | Train.Rand 1.0000 | Valid.Rand 0.5818 | Time(s) 0.1799


/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 00174 | Loss 0.0766 | Train.Rand 0.9922 | Valid.Rand 0.6024 | Time(s) 0.1799
Epoch 00175 | Loss 0.1038 | Train.Rand 0.9813 | Valid.Rand 0.5890 | Time(s) 0.1799


<All keys matched successfully>

In [20]:
net.load_state_dict(current_best_params)

<All keys matched successfully>

In [21]:
np.unique(np.argmax(logits[mask_train].detach().numpy(),1))

array([0, 1, 2, 3, 4, 5, 6])

In [61]:
# Visualise predictions
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(features).detach()
a = np.transpose(np.vstack([final_prediction[comb_mask].numpy().argmax(axis=1),labels[comb_mask].numpy()]))
a[a[:,0].argsort()][np.random.choice(range(a.shape[0]),size=10)]

# As can be seen, the graph net predicts correct labels

array([[2, 2],
       [2, 2],
       [2, 2],
       [3, 3],
       [0, 0],
       [2, 2],
       [0, 0],
       [3, 3],
       [3, 3],
       [4, 4]])

## Evaluation


In [62]:
# Performnace evaluation functions

def variation_of_information_score(labels, preds):
    def mi(x, y):
        contingency = skmetrics.cluster.contingency_matrix(x, y, sparse=True)
        # print(contingency.todense())
        nzx, nzy, nz_val = sp.find(contingency)
        contingency_sum = contingency.sum()

        pi = np.ravel(contingency.sum(axis=1))
        pj = np.ravel(contingency.sum(axis=0))
        # print(nz_val)
        log_contingency_nm = np.log(nz_val)
        # print(log_contingency_nm)
        contingency_nm = nz_val / contingency_sum
        # print(contingency_nm)

        # Don't need to calculate the full outer product, just for non-zeroes
        outer = pi.take(nzx).astype(np.int64, copy=False) * pj.take(nzy).astype(
            np.int64, copy=False
        )
        # print(outer)
        log_outer = -np.log(outer) + log(pi.sum()) + log(pj.sum())
        # print(log_outer)
        mi = (
            contingency_nm * (log_contingency_nm - log(contingency_sum))
            + contingency_nm * log_outer
        )
        # print(mi)
        return mi.sum()

    return mi(labels, labels) + mi(preds, preds) - 2 * mi(labels, preds)


def mutual_info_score(labels, preds):
    return skmetrics.adjusted_mutual_info_score(labels, preds, average_method="arithmetic")

def compute_performance(labels, logits, splits):
    logits = logits.detach().numpy()
    preds = np.argmax(logits, axis=1)
    labels = labels.numpy()
    pred_sets = {
        "All ": preds,
        "Train": preds[mask_train],
        "Val": preds[mask_val],
        "Test": preds[test_mask],
    }
    label_sets = {
        "All ": labels,
        "Train": labels[mask_train],
        "Val": labels[mask_val],
        "Test": labels[test_mask],
    }
    eval_functions = {
        "Rand-Index": rand_score,
        "Mutual Information": mutual_info_score,
        "Variation of Information": variation_of_information_score,
    }
    scores = {
        subset: {
            name: func(label_sets[subset], pred_sets[subset])
            for name, func in eval_functions.items()
        }
        for subset in pred_sets.keys()
    }
    return scores

def performance_as_df(labels, logits, splits):
    scores = compute_performance(labels, logits, splits)
    return pd.DataFrame(scores)

def rand_score(labels, preds):
    return skmetrics.adjusted_rand_score(labels, preds)

In [64]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(features).detach()
performance_as_df(labels,final_prediction,comb_mask)

,All,Train,Val,Test
Rand-Index,0.627315,1.0,0.573450,0.611194
Mutual Information,0.607157,1.0,0.579221,0.592505
Variation of Information,1.436844,0.0,1.465711,1.472488
